Preparing the data

In [12]:
# !pip install numdifftools
import numpy as np
import pandas as pd
from scipy.stats import norm

data = pd.read_csv("data425288.csv")

T = 25
N = data.ID.max()

y = np.log(data.Sales.values.reshape((N,T)))

X = np.ones((T,2))
X[:,1] = np.log(data.Price[:T])

OLS for reference

In [3]:
import statsmodels.formula.api as sm

regdata = data.copy()
regdata["logS"] = np.log(regdata.Sales)
regdata["logP"] = np.log(regdata.Price)
result = sm.ols(formula="logS ~ logP", data=regdata).fit()
display(result.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   logS   R-squared:                       0.261
Model:                            OLS   Adj. R-squared:                  0.261
Method:                 Least Squares   F-statistic:                     4408.
Date:                Mon, 09 Dec 2019   Prob (F-statistic):               0.00
Time:                        21:56:06   Log-Likelihood:                -19197.
No. Observations:               12500   AIC:                         3.840e+04
Df Residuals:                   12498   BIC:                         3.841e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     10.7127      0.023    462.944      0.000      10.667      10.758
logP          -1.3974      0.021    -66.393      0.000      -1.439      -1.356
==============================================================================
Omnibus:                        1.098   Durbin-Watson:                   1.596
Prob(Omnibus):                  0.577   Jarque-Bera (JB):                1.113
Skew:                          -0.002   Prob(JB):                        0.573
Kurtosis:                       2.954   Cond. No.                         4.40
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Log-likelihood evaluation

In [4]:
def LogL(theta, pi, y, X):
    K=theta.shape[0]
    mu = np.dot(X,theta.T) #(25, 3)
    mu = np.repeat(mu[np.newaxis, :, :], N, axis=0)  #(500,25,K)
    y = np.repeat(y[:, :, np.newaxis], K, axis=2)  #(500,25,K)
    
    #pdfs
    probs = norm.pdf(y,mu,1) #(500,25,K)
    
    #prod^T probs
    segments = np.prod(probs, axis=1) #(500,K)
    
    #sum^K pi_c prod^T probs
    combined = np.dot(segments, pi) #(500,)
    
    #sum^N log sum^K pi_c prod^T probs
    LogL = np.log(combined).sum() #(1,)
    return LogL

Expectation step

In [5]:
def EStep(theta,pi,y,X, robust = True):
    K=theta.shape[0] 
    mu = np.dot(X,theta.T) #(25, K)
    mu = np.repeat(mu[np.newaxis, :, :], N, axis=0)  #(500,25,K)
    y = np.repeat(y[:, :, np.newaxis], K, axis=2)  #(500,25,K)

    #pdfs
    probs = norm.pdf(y,mu,1) #(500,25,K)
    
    #if to be calcluted by exp(log()) to prevent underflow
    if robust:
        #prod^T probs as sum^T log p_t to prevent small numbers
        segments = np.log(probs).sum(axis=1)
        
        # prod^T probs times diagonal of pi as exp(sum^T log p_t + log pi_c - most negative number)
        numerators = np.exp(segments + np.log(pi) - segments.min())    
    
    #using the direct definition
    else:
        #prod^T probs
        segments = np.prod(probs, axis=1) #(500,K)
        
        # prod^T probs times diagonal of pi
        numerators = np.dot(segments, np.diag(pi))
    
    #divide numerators by denominators (= sum of row)
    W = numerators / numerators.sum(axis=1, keepdims=True)

    return W

Maximization step

In [6]:
def MStep(W,y,X):
    K=W.shape[1]
    
    # 1/N sum^N w_ic
    pi = W.mean(axis=0) #(K,)
    
    theta = np.zeros((K,2))
    
    for c in range(K):        
        inv = np.zeros((2,2))
        for i in range(N): 
            inv += W[i,c] * np.dot(X.T, X) 
        inv  =  np.linalg.inv(inv)
        
        theta[c,:] = np.linalg.multi_dot([inv,X.T,y.T, W[:,c]])
    
    return theta, pi

EM algorithm

In [7]:
def EM(K,y,X,tolerance=0.0001, verbose=0):
    #W initialized randomly
    W = np.random.rand(N,K)
    W = W/W.sum(axis=1, keepdims=True)
    theta = None
    pi = None
    iterations=0
    max_iter = 250
    
    diff = np.inf
    while (diff >= tolerance and iterations < max_iter):
        iterations+=1
        theta, pi = MStep(W,y,X)
        currentW = EStep(theta,pi,y,X, robust = True)
        diff = np.linalg.norm(W-currentW)
        W = currentW
        
        if verbose==2:            
            print("Iteration "+str(iterations))
            print("Log-likelihood: "+str(LogL(theta, pi, y, X)))
            print("\n")
    
    if(iterations == max_iter): print("Maximum iterations reached, iters: ",max_iter)
    if verbose >= 1:
        print("Log-likelihood: "+str(LogL(theta, pi, y, X)))
    
    return theta, pi

Estimation implementation

In [72]:
import numdifftools as nd

def Estimate(K, X=X, y=y, tolerance=0.1, seed=1234,verbose=2, n_est=10):
    theta = None
    pi = None
    Likelihood = -np.inf
    np.random.seed(seed)
    
    #performing the estimations
    for i in range(n_est):
        if verbose >= 1: print("\nEstimation ",i+1)
            
        t, p = EM(K,y,X,tolerance=tolerance, verbose=verbose)
        currentLogL = LogL(t, p, y, X)
            
        if currentLogL >= Likelihood:
            theta = t
            pi = p
            Likelihood = currentLogL
    
    if verbose >= 1:
        print("The estimation is finished")
        print("\nLog likelihood: "+str(Likelihood))
        print("\nTheta:")
        print(theta)
        print("\nPi:")
        print(pi)
    
    
    #calcutating the standard errors
    gamma = np.zeros(K)
    for c in range(K-1):
        gamma[c] = np.log(pi[c]) - np.log(pi[K-1])
        
    def hes_eval(input):
        theta = input[:K*2].reshape(K,2)
        gamma = input[K*2:]

        #Restricted Log Likelihood implementation
        pi = np.exp(gamma) / np.exp(gamma).sum()
        return LogL(theta, pi, y, X)
            
    input = np.concatenate((theta, gamma), axis=None)
    hessian = nd.Hessian(hes_eval)(input)
    stderrors = np.sqrt(np.linalg.inv(-1 * hessian).diagonal())
    
    theta_std = stderrors[:K*2].reshape(K,2)
    pi_std = stderrors[K*2:]
    
    #preparing results
    results = [{
            "p": "LogL",
            "value": Likelihood,
            "K": K
        }]
    
    for c in range(K):
        results += [{
            "p": "α "+str(c+1),
            "value": "%.3f (%.3f)" % (theta[c,0], theta_std[c,0]),
            "K": K
        },{
            "p": "β "+str(c+1),
            "value": "%.3f (%.3f)" % (theta[c,1], theta_std[c,1]),
            "K": K
        },{
            "p": "π "+str(c+1),
            "value": "%.3f" % (pi[c]),
            "K": K
        }]

        
    return results

Performing the estimations

In [89]:
tol = 0.1
n_estimations = 1

table = pd.DataFrame.from_dict(
     Estimate(K=2, X=X, y=y, tolerance=tol, verbose=1, n_est=n_estimations)
    +Estimate(K=3, X=X, y=y, tolerance=tol, verbose=1, n_est=n_estimations)
    +Estimate(K=4, X=X, y=y, tolerance=tol, verbose=1, n_est=n_estimations)
    +Estimate(K=5, X=X, y=y, tolerance=tol, verbose=1, n_est=n_estimations)
)

pivot = table.pivot(index='p', columns='K', values='value')
display(pivot)
print(pivot.to_latex())


Estimation  1
Log-likelihood: -18133.691829702675

Estimation  2
Log-likelihood: -18133.691164130305

Estimation  3
Log-likelihood: -18133.69038213583

Estimation  4
Log-likelihood: -18133.690285224075

Estimation  5
Log-likelihood: -18133.690487852782
The estimation is finished

Log likelihood: -18133.690285224075

Theta:
[[11.19631796 -1.44210983]
 [10.08142094 -1.33893974]]

Pi:
[0.56621491 0.43378509]


C:\Users\Tim\Anaconda3\lib\site-packages\ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in sqrt



Estimation  1
Log-likelihood: -18130.31096179693

Estimation  2
Log-likelihood: -18115.849623657003

Estimation  3
Log-likelihood: -18129.381336631955

Estimation  4
Log-likelihood: -18115.84497787928

Estimation  5
Log-likelihood: -18115.860315839294
The estimation is finished

Log likelihood: -18115.84497787928

Theta:
[[10.49008474 -0.8055812 ]
 [10.08264554 -1.34081488]
 [11.3466012  -1.57795353]]

Pi:
[0.10081436 0.43321122 0.46597442]

Estimation  1
Log-likelihood: -18111.160219193473

Estimation  2
Log-likelihood: -18111.252510558905

Estimation  3
Log-likelihood: -18128.61701323693

Estimation  4
Log-likelihood: -18111.51990684928

Estimation  5
Log-likelihood: -18111.659047077308
The estimation is finished

Log likelihood: -18111.160219193473

Theta:
[[11.35710934 -1.57731775]
 [10.48885881 -0.79715795]
 [10.54390523 -1.521092  ]
 [10.00584491 -1.30786624]]

Pi:
[0.45436157 0.09933827 0.08345339 0.36284677]

Estimation  1
Log-likelihood: -18110.636229757038

Estimation  2
Log

C:\Users\Tim\Anaconda3\lib\site-packages\ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in sqrt


K,2,3,4,5
p,,,,
LogL,-18133.7,-18115.8,-18111.2,-18109.2
α 1,11.196 (0.028),10.490 (0.171),11.357 (0.050),10.414 (0.162)
α 2,10.081 (0.032),10.083 (0.032),10.489 (0.169),9.991 (0.117)
α 3,NaN,11.347 (0.049),10.544 (0.246),10.454 (0.338)
α 4,NaN,NaN,10.006 (0.074),11.281 (0.157)
α 5,NaN,NaN,NaN,11.368 (0.092)
β 1,-1.442 (0.025),-0.806 (0.158),-1.577 (0.044),-0.730 (0.147)
β 2,-1.339 (0.029),-1.341 (0.029),-0.797 (0.154),-1.299 (0.075)
β 3,NaN,-1.578 (0.044),-1.521 (0.109),-1.504 (0.123)


\begin{tabular}{lllll}
\toprule
K &               2 &               3 &               4 &               5 \\
p    &                 &                 &                 &                 \\
\midrule
LogL &        -18133.7 &        -18115.8 &        -18111.2 &        -18109.2 \\
α 1  &  11.196 (0.028) &  10.490 (0.171) &  11.357 (0.050) &  10.414 (0.162) \\
α 2  &  10.081 (0.032) &  10.083 (0.032) &  10.489 (0.169) &   9.991 (0.117) \\
α 3  &             NaN &  11.347 (0.049) &  10.544 (0.246) &  10.454 (0.338) \\
α 4  &             NaN &             NaN &  10.006 (0.074) &  11.281 (0.157) \\
α 5  &             NaN &             NaN &             NaN &  11.368 (0.092) \\
β 1  &  -1.442 (0.025) &  -0.806 (0.158) &  -1.577 (0.044) &  -0.730 (0.147) \\
β 2  &  -1.339 (0.029) &  -1.341 (0.029) &  -0.797 (0.154) &  -1.299 (0.075) \\
β 3  &             NaN &  -1.578 (0.044) &  -1.521 (0.109) &  -1.504 (0.123) \\
β 4  &             NaN &             NaN &  -1.308 (0.048) &  -1.624 (0.126) \\
β 